<a href="https://colab.research.google.com/github/deokwoo-han/abc/blob/master/project3_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
color = sns.color_palette()

%matplotlib inline
%config InlineBackend.figure_config = 'retina'

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pwd cd

/content


In [ ]:
file_path = './drive/MyDrive/datas'
order_products_train = pd.read_csv(file_path + "/order_products__train.csv")
order_products_prior = pd.read_csv(file_path + "/order_products__prior.csv")
orders = pd.read_csv(file_path + "/orders.csv")
products = pd.read_csv(file_path + "/products.csv")
aisles = pd.read_csv(file_path + "/aisles.csv")
departments = pd.read_csv(file_path + "/departments.csv")

In [ ]:
order_products_prior = pd.merge(order_products_prior, products, on='product_id', how='left')
order_products_prior = pd.merge(order_products_prior, aisles, on='aisle_id', how='left')
order_products_prior = pd.merge(order_products_prior, departments, on='department_id', how='left')
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry


In [ ]:
df2 = pd.DataFrame(order_products_prior.groupby('order_id').department.agg(lambda x: x.tolist())).reset_index()
df2.head(10)

,order_id,department
0,2,"[dairy eggs, produce, pantry, pantry, pantry, ..."
1,3,"[dairy eggs, dairy eggs, produce, produce, dai..."
2,4,"[bakery, personal care, snacks, breakfast, bre..."
3,5,"[produce, pantry, deli, produce, dairy eggs, h..."
4,6,"[beverages, household, household]"
5,7,"[beverages, frozen]"
6,8,[bakery]
7,9,"[produce, produce, produce, dairy eggs, snacks..."
8,10,"[produce, produce, produce, produce, produce, ..."
9,11,"[frozen, deli, canned goods, snacks, pantry]"


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
te = TransactionEncoder() 
te_result = te.fit(df2.department).transform(df2.department)
df2_1 = pd.DataFrame( te_result, columns = te.columns_)
df2_1.head()

,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,frozen,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False
1,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False
2,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
3,False,False,False,True,False,False,False,True,True,True,False,True,True,True,False,False,True,True,False,True,True
4,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [ ]:
itemset1 = apriori(df2_1, use_colnames=True, min_support=0.03) 

In [ ]:
itemset1

,support,itemsets
0,0.055278,(babies)
1,0.274212,(bakery)
2,0.453315,(beverages)
3,0.163362,(breakfast)
4,0.211923,(canned goods)
...,...,...
718,0.031293,"(produce, dry goods pasta, snacks, dairy eggs,..."
719,0.045762,"(produce, pantry, snacks, dairy eggs, frozen, ..."
720,0.032411,"(produce, pantry, snacks, dairy eggs, frozen, ..."
721,0.034251,"(produce, pantry, snacks, dairy eggs, frozen, ..."


In [ ]:
df2_2 = association_rules(itemset1, metric="confidence", min_threshold=0.85 )
df2_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(canned goods),(produce),0.211923,0.749429,0.186306,0.879123,1.173057,0.027485,2.072944
1,(deli),(produce),0.239605,0.749429,0.206059,0.859995,1.147533,0.026492,1.789726
2,(dry goods pasta),(produce),0.185967,0.749429,0.161743,0.869737,1.160533,0.022373,1.923585
3,(international),(produce),0.068910,0.749429,0.059938,0.869805,1.160624,0.008295,1.924583
4,(meat seafood),(produce),0.178772,0.749429,0.156475,0.875276,1.167923,0.022498,2.008996
...,...,...,...,...,...,...,...,...,...
561,"(pantry, snacks, dairy eggs, frozen, deli)",(produce),0.036712,0.749429,0.034251,0.932971,1.244909,0.006738,3.738261
562,"(pantry, snacks, deli, frozen)","(produce, dairy eggs)",0.040275,0.550483,0.034251,0.850425,1.544869,0.012080,3.005287
563,"(produce, dry goods pasta, pantry, snacks, fro...",(dairy eggs),0.033124,0.677270,0.030741,0.928076,1.370319,0.008308,4.487121
564,"(dry goods pasta, snacks, dairy eggs, pantry, ...",(produce),0.033143,0.749429,0.030741,0.927545,1.237669,0.005903,3.458301
